# CoronaInDeStad.nl Web Crawler

Web crawler for the website coronaindestad.nl

In [84]:
from bs4 import BeautifulSoup
import requests
import time

In [3]:
URL = "https://www.coronaindestad.nl/small-poetic-sonic-events-2/"

In [4]:
web_page = requests.get(URL)

In [67]:
def get_next_link(web_page, target_class="next-button"):
    page_links = []
    for a in BeautifulSoup(web_page, "html.parser").select('a'):
        a_class = a.get("class")
        if type(a_class) == list and target_class in a_class:
            page_links.append(a.get("href"))
    return page_links

In [71]:
def get_text(web_page, target_class="large-7"):
    text = []
    for div in BeautifulSoup(web_page, "html.parser").select('div'):
        div_class = div.get("class")
        if type(div_class) == list and target_class in div_class:
            text.append(div.text)
    return text

In [80]:
next_url = get_next_link(web_page.content)[0]

In [81]:
text = get_text(web_page.content)[-1]

In [82]:
print(next_url)

https://www.coronaindestad.nl/empty-city/?h=


In [83]:
print(text)


May 13th 2020
Passeerdersgracht morning:
Meerkoeten, seagulls, small birds in the tree outside the window, the leaves of the tree, one set of footsteps crossing the bridge on the Prinsengracht, some wind, a man laughing far away, one car and another on the Marnixstraat, a bicycle creaking, boats rocking in the water.
Lijbaansgracht surprise:
For several days there is the sound of water falling about a meter, out of a building and into the canal.
Passeerdersgracht days:
Every time there is an airplane, I notice it.
A man and a woman dressed in formal attire serenading a house further down the canal. They do this twice in one week.
A father and two small children playing on their stoop all afternoon.
Staying Alive blasting from the windows of the apartment next door
The business owner downstairs sanding his hallway floor at 7am. It is very loud.
A motorbike that 2 days in a row rides around and around like a racetrack
People sitting on the pontje talking in loud American voices about on

In [86]:
start_time = time.time()
next_web_page = requests.get(next_url)
print(int(time.time() - start_time))

129
